In [10]:
import pyzx as zx
import fractions

state = zx.Graph()
out1 = state.add_vertex(zx.VertexType.BOUNDARY, 0, 1)
out2 = state.add_vertex(zx.VertexType.BOUNDARY, 1, 1)
in1 = state.add_vertex(zx.VertexType.X, 0, 0)
in2 = state.add_vertex(zx.VertexType.X, 1, 0)
state.add_edges([(in1, out1), (in2, out2)])
state.set_outputs([out1, out2])
state.set_inputs([])

circ1 = zx.qasm("""
qreg qubits[2];

h qubits[0];
cx qubits[0], qubits[1];
s qubits[0];
""").to_graph()

circ2 = zx.qasm("""
qreg qubits[2];

h qubits[0];
cx qubits[0], qubits[1];
s qubits[0];
z qubits[0];
x qubits[0];
x qubits[1];
""").to_graph()

x = zx.Scalar()
x.phase = fractions.Fraction(1, 2)
circ2.scalar.mult_with_scalar(x)

zx.draw(circ1 * state)
zx.draw(circ2 * state)

display(zx.compare_tensors(circ1 * state, circ2 * state, preserve_scalar=True))

display(zx.print_matrix(circ1 * state))
display(zx.print_matrix(circ2 * state))

True

Label(value='\\begin{equation}\n\\sqrt{2}\n\\begin{pmatrix}\n1 \\\\ \n0 \\\\ \n0 \\\\ \ni\n\\end{pmatrix}\n\\e…

Label(value='\\begin{equation}\n\\sqrt{2}\n\\begin{pmatrix}\n1 \\\\ \n0 \\\\ \n0 \\\\ \ni\n\\end{pmatrix}\n\\e…

In [4]:
def id(len: int):
    return zx.qasm(f"qreg x[{len}];").to_graph()

def cnot(len: int, control: int, target: int):
    return zx.qasm(f"qreg x[{len}]; cx x[{control}], x[{target}];").to_graph()

def had(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}];").to_graph()

def t(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; t x[{qubit}];").to_graph()

def tdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; tdg x[{qubit}];").to_graph()

def s(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; s x[{qubit}];").to_graph()

def sdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; sdg x[{qubit}];").to_graph()

def rootx(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}]; s x[{qubit}]; h x[{qubit}];").to_graph()

def rootxdg(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; h x[{qubit}]; sdg x[{qubit}]; h x[{qubit}];").to_graph()

def z(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; z x[{qubit}];").to_graph()

def x(len: int, qubit: int):
    return zx.qasm(f"qreg x[{len}]; x x[{qubit}];").to_graph()
 

def create_pauli(string: str, sign: bool = True, gate_pos = t, gate_neg = tdg):
    base = id(len(string))

    for i in range(len(string)):
        if string[i] == "X":
            base = had(len(string), i) * base
        elif string[i] == "Y":
            base = rootx(len(string), i).adjoint() * base

    
    base = create_phase_poly(string.replace("X", "Z").replace("Y", "Z"), sign, 0, gate_pos, gate_neg) * base

    for i in range(len(string)):
        if string[i] == "X":
            base = had(len(string), i) * base
        elif string[i] == "Y":
            base = rootx(len(string), i) * base

    return base


def create_phase_poly(string: str, sign: bool = True, i: int = 0, gate_pos = t, gate_neg = tdg):
    if i >= len(string):
        return id(len(string))
    rec = create_phase_poly(string, sign, i + 1, gate_pos, gate_neg)
    if string[i] == "I":
        return rec
    next_i = i + 1
    while next_i < len(string) and string[next_i] == "I":
        next_i = next_i + 1
    if next_i >= len(string):
        if sign:
            return gate_pos(len(string), i)
        else:
            return gate_neg(len(string), i)
    return cnot(len(string), i, next_i) * rec * cnot(len(string), i, next_i)

control = "Z"
string = "XZIYZ"

control = string.index(control)
target = 4

graph = cnot(5, control, target) * create_pauli(string)

simp_graph = create_pauli("XIIYZ") * cnot(5, control, target)

display(zx.print_matrix(graph))
display(zx.print_matrix(simp_graph))

display(zx.compare_tensors(graph, simp_graph))



    

Label(value='\\begin{equation}\n0.92e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0…

Label(value='\\begin{equation}\n0.92e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0…

True

In [12]:
pauli = create_pauli("ZY")
state1 = pauli * had(2, 0) * state
state2 = x(2, 0) * z(2, 1) * state1

zx.draw(state1)
zx.draw(state2)
display(zx.print_matrix(state1))

zx.print_matrix(state2)

Label(value='\\begin{equation}\n1.31e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 \\\\ \n1-\\sqrt{2} \\\\ \n1 \\\…

Label(value='\\begin{equation}\n1.31e^{i\\frac{1}{8}\\pi}\n\\begin{pmatrix}\n1 \\\\ \n1-\\sqrt{2} \\\\ \n1 \\\…

In [5]:
import math

def compare_tensors(ta, tb, verbose=False):
    ma = -math.inf
    mai = -1
    mb = -math.inf
    mbi = -1
    for j,x in enumerate(ta.flat):
        if x > ma:
            ma = x
            mai = j
    for j,x in enumerate(tb.flat):
        if x > mb:
            mb = x
            mbi = j
    max_i = mai if ma > mb else mbi
    epsilon = 1e-8
    for i in range(len(ta.flat)):
        diff = abs((ta.flat[i] / ta.flat[max_i]) - (tb.flat[i] / tb.flat[max_i]))
        if diff > epsilon:
            if verbose:
                print(diff)
                print(i)
                print(ta.flat[i], tb.flat[i])
                print(max_i)
                print(ta.flat[max_i], tb.flat[max_i])
                print((ta.flat[i] / ta.flat[max_i]), (tb.flat[i] / tb.flat[max_i]))
            return False


    return True
    
    

def compare_circs(ga, gb, verbose=False):
    ta = ga.to_tensor(preserve_scalar=False)
    tb = gb.to_tensor(preserve_scalar=False)
    return compare_tensors


def print_tensor_pair(ta, tb):
    for j,x in enumerate(ta.flat):
        print(ta.flat[j], tb.flat[j])


In [4]:
import pyzx as zx

def new_state(qubits: int):
  s = zx.Graph()
  outs = []
  for i in range(qubits):
    outs.append(s.add_vertex(zx.VertexType.BOUNDARY, i, 1))
    in1 = s.add_vertex(zx.VertexType.X, i, 0)
    s.add_edge((in1, outs[-1]))
  s.set_outputs(outs)
  s.set_inputs([])
  return s

file = open("qasm_files/test.qasm", "r+")
file2 = open("test.qasm", "r+")
lines = "\n".join(file.readlines())
lines2 = "\n".join(file2.readlines())


a = zx.qasm(lines)
b = zx.qasm(lines2)

state = new_state(a.qubits)

a = a.to_graph() * state
b = b.to_graph() * state

zx.full_reduce(a)
# zx.full_reduce(b)

display(zx.print_matrix(a))
display(zx.print_matrix(b))


zx.draw(a)
zx.draw(b)

# ta = a.to_tensor(preserve_scalar=False)

print(zx.compare_tensors(a, b, False))



Label(value='\\begin{equation}\n(-0.05+0.17i)10^{-6}\n\\begin{pmatrix}\n(-1.42-0.76i)10^{7} \\\\ \n1 \\\\ \n-0…

Label(value='\\begin{equation}\n\\sqrt{2}^{3}\n\\begin{pmatrix}\n1 \\\\ \n0 \\\\ \n0 \\\\ \n0 \\\\ \n0 \\\\ \n…

False


In [20]:

# tb = b.to_tensor(preserve_scalar=False)

In [2]:
# print(compare_tensors(ta, tb, True))
print(zx.compare_tensors(a, b, False))
# print_tensor_pair(ta, tb)


True


In [ ]:
import itertools

def powerset(l):
    for sl in itertools.product(*[[[], [i]] for i in l]):
        yield {j for i in sl for j in i}

stabs = ["XZIZXIIIIIIIIZIZ",
"ZXYZZZXIIIIIIYZZ",
"ZZXZIZXIIIIIIYZZ",
"ZZZXYZZZXIIIIIIY",
"ZZZZXZIZXIIIIIIY",
"IIIZIXYZZZXIIIII",
"IIIZIZXZIZXIIIII",
"IIIIIZIXYZZZXIII",
"IIIIIZIZXZIZXIII",
"IIIIIIIZIXYZZZXI",
"IIIIIIIZIZXZIZXI",
"ZIZIIIIIIIIYZIZY",
"ZIZIIIIIIZIZXIIY",
"ZIIIIIIIIIIIIYZI",
"IIIIIIIIIIIZIZXI",
"ZIZIIIIIIIIIIIIY"]

def add_strings(pauli1, pauli2):
  result = ""
  for i in range(len(pauli1)):
    if pauli1[i] == "I": result += pauli2[i]; continue
    if pauli2[i] == "I": result += pauli1[i]; continue
    if pauli1[i] == pauli2[i]: result += "I"; continue
    if pauli1[i] == "X":
      if pauli2[i] == "Y": result += "Z"; continue
      if pauli2[i] == "Z": result += "Y"; continue

    if pauli1[i] == "Y":
      if pauli2[i] == "X": result += "Z"; continue
      if pauli2[i] == "Z": result += "X"; continue

    if pauli1[i] == "Z":
      if pauli2[i] == "Y": result += "X"; continue
      if pauli2[i] == "X": result += "Y"; continue
  return result

def do_commute(pauli1, pauli2):
  result = 0
  for i in range(len(pauli1)):
    if pauli1[i] == "I" or pauli2[i] == "I": continue
    if pauli1[i] != pauli2[i]: result += 1
  return result % 2 == 0


for x in stabs:
  for y in stabs:
    assert(do_commute(x, y))


merge1 = "IXXZIZXIIIIIIYZY"
merge2 = "YZIZXIIIIIIIIYZZ"

print(add_strings(merge1, merge2))

to_commute = "YYXIXZXIIIIIIIIX"
output     = "ZYXIXZXIIIIIIIIY"

blocker    = "IYXIXZXIIIIIIYZY"

print(add_strings("ZIIIIIIIIIIIIYZI", blocker))

assert(not do_commute(to_commute, blocker))
assert(do_commute("ZIIIIIIIIIIIIYZI", blocker))
assert(do_commute(to_commute, add_strings("ZIIIIIIIIIIIIYZI", blocker)))

contains = 0

for stab_set in powerset(stabs):
  base = "I" * 16
  for stab in stab_set:
    base = add_strings(base, stab)
  if not do_commute(blocker, base): continue
  base = add_strings(blocker, base)
  if do_commute(to_commute, base):
    # print(stab_set)
    if "ZIIIIIIIIIIIIYZI" in stab_set:
      contains += 1
    # break

print(contains)


YYXIXZXIIIIIIIIX
ZYXIXZXIIIIIIIIY
8192


In [ ]:
stabs = [
"ZIIIIIII",
"IZIIIIII",
"IIZIIIII",
"IIIZIIII",
"IIIIZIII",
"IIIIIZII",
"IIIIIIZI",
"IIIIIIIZ"]

to_commute = "ZIZIIIII"
blocker = "XYIIIIII"


for stab_set in powerset(stabs):
  base = "I" * 8
  for stab in stab_set:
    base = add_strings(base, stab)
  if not do_commute(blocker, base): continue
  base = add_strings(blocker, base)
  if do_commute(to_commute, base):
    print(stab_set)
    # break



In [25]:
single_cnot = cnot(2, 0, 1)
pauli_cnot = create_pauli("ZX", False, s, sdg) * create_pauli("ZI", True, s, sdg) * create_pauli("IX", True, s, sdg)

zx.draw(pauli_cnot)
zx.draw(single_cnot)

display(zx.print_matrix(single_cnot))
display(zx.print_matrix(pauli_cnot))

zx.compare_tensors(single_cnot, pauli_cnot, preserve_scalar=True)


Label(value='\\begin{equation}\n\n\\begin{pmatrix}\n1 & 0 & 0 & 0 \\\\ \n0 & 1 & 0 & 0 \\\\ \n0 & 0 & 0 & 1 \\…

Label(value='\\begin{equation}\n\n\\begin{pmatrix}\n1 & 0 & 0 & 0 \\\\ \n0 & 1 & 0 & 0 \\\\ \n0 & 0 & 0 & 1 \\…

True